In [262]:
import requests
import pandas as pd
import datetime
from functools import wraps

In [263]:
# Decorators
def error_log(function_name):
    def inner_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print(f"Function {function_name} encountered an error: {e}")
                
        return wrapper
    return inner_decorator

In [79]:
# @error_log('fetch_data')
def fetch_data(url):

    # call API
    r = requests.get(url)
    op = r.json()
    
    # json to data frame
    df = pd.json_normalize(op['response']['data'])
    df = df.pivot(columns="type-name", values='value', index = ['period', 'respondent'] )
    
    df.reset_index(inplace=True)
    
    # organize data as needed
    df.drop(columns=['Day-ahead demand forecast','Total interchange'], inplace=True)
    df[['Date', 'Hour']] = df['period'].str.split('T',expand=True)
    df = df.drop('period', axis = 1)
    df.rename(columns={'respondent':'region'}, inplace=True)
    df = df[['Date','Hour','region','Demand','Net generation']]
    
    # segregate data based on regions
    final_dict = segregate_to_region(df)

    return final_dict

In [229]:
# @error_log('segregate_to_region')
def segregate_to_region(df):
    data = {}
    for r in regions:
        data[r] = df[df['region']==r].reset_index().drop(columns=['index'])
        if len(data[r])<96:
            temp = data_correction(r,data[r])
            temp.reset_index(inplace=True)
            temp.drop(columns=['index'],inplace=True)
            data[r] = temp
        
    return data

In [105]:
# @error_log('data_correction')
def data_correction(r,df):
    hour = ['{:02d}'.format(i) for i in range(24)]
    uq_date = df['Date'].unique().tolist()
    dfs=[]
    for dt in uq_date:
        uq_hour = df[df['Date']==dt]['Hour'].unique().tolist()
        missing_hr = [x for x in hour if x not in uq_hour]
        if len(missing_hr)!=0:
            new_df=generate_data(df[(df['Date']==dt)],missing_hr,r,dt)
#             print(r,dt,missing_hr,len(new_df))
            dfs.append(new_df)
        else:
            dfs.append(df[(df['Date']==dt)])
    print(uq_date)
    dfs = pd.concat(dfs)
#     print(len(dfs))
    return dfs

In [82]:
# @error_log('generate_data')
def generate_data(df,missing_hr,r,dt):
    
    demand_median = df['Demand'].median()
    net_gen_median = df['Net generation'].median()
    
    new_data=[]
    for hr in missing_hr:
        new_data.append((dt,hr,r,demand_median,net_gen_median))
        
    df_missing = pd.DataFrame(new_data,columns=['Date', 'Hour', 'region','Demand','Net generation'])
    df_new = pd.concat([df,df_missing])
    df_new = df_new.sort_values(['Date','Hour'])
    
    return df_new

In [83]:
# @error_log('get_new_dates')
def get_new_dates(start_date,end_date):
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    st =  end_date + datetime.timedelta(days=1)
    start_date = st.strftime('%Y-%m-%d')
    end_date = (st + datetime.timedelta(days=3)).strftime('%Y-%m-%d')
    
    return start_date, end_date

In [235]:
start_date =  "2015-07-01"
end_date = "2015-07-04"
regions = ['CAL', 'CAR', 'CENT', 'FLA', 'MIDA', 'MIDW', 'NE', 'NY', 'SE', 'SW', 'TEN', 'TEX']
region_data_dict={}

In [ ]:
while end_date<='2023-03-15':
    
    url= f"https://api.eia.gov/v2/electricity/rto/region-data/data/?\
frequency=hourly&data[0]=value&facets[respondent][]=CAL\
&facets[respondent][]=CAR&facets[respondent][]=CENT&facets[respondent][]=FLA&facets[respondent][]=MIDA\
&facets[respondent][]=MIDW&facets[respondent][]=NE&facets[respondent][]=NY&facets[respondent][]=SE&\
facets[respondent][]=SW&facets[respondent][]=TEX&\
start={start_date}T00&end={end_date}T23\
&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000\
&api_key=2Ztw7IK10RqAv0oag9T2o2FOV8YZgRpapfTEvhwH"
    print(f'Now fetching data from {start_date} to {end_date}')
    temp=fetch_data(url)
    ct=0
    for k in temp.keys():
        if k in region_data_dict.keys():
            region_data_dict[k].append(temp[k])
            ct+=len(temp[k])
        else:
            region_data_dict[k]=[temp[k]]
            ct+=len(temp[k])

    print(f'Got demand data from date {start_date} to {end_date} and its count is {ct}')
    
    start_date,end_date = get_new_dates(start_date,end_date)

In [239]:
region_data_dict_copy = dict(region_data_dict)

In [240]:
for k in region_data_dict_copy.keys():
    region_data_dict_copy[k]=pd.concat(region_data_dict_copy[k])

In [246]:
for k in region_data_dict_copy.keys():
    region_data_dict_copy[k].to_csv(f'{k}--2015-07-01--2023-03-15.csv',index=False)